# Video Vision Transformer Model Implementation v0

__Author:__ Jack Friedman <br>
__Date:__ 11/15/2024 <br>
__Adapted from:__ Aritra Roy Gosthipaty and Ayush Thakur (https://github.com/keras-team/keras-io/blob/master/examples/vision/vivit.py) <br>
__Original Paper:__ ViViT: A Video Vision Transformer (https://arxiv.org/abs/2103.15691) by Arnab et al. <br>
__Updates from v0:__ 
- Preprocessing_v4 - clip output, remove height from tensors
- Added regularization, dropout, and early stopping
- Changed model to be same output format as Zoo model

## Import libraries

In [7]:
import os
import io
import time 
from datetime import timedelta
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras import layers, regularizers, callbacks
from keras import backend as K
from keras.saving import register_keras_serializable
from keras.models import load_model, save_model
from sklearn.model_selection import train_test_split
import sys
sys.path.append('../preprocessing')
from Preprocessing_v4 import *
from DataLoader import load_data

In [2]:
# Setting seed for reproducibility
SEED = 42
os.environ["TF_CUDNN_DETERMINISTIC"] = "1"
keras.utils.set_random_seed(SEED)

## Step 0: Define key hyperparameters and constants

In [3]:
"""
Code authors tuned these hyperparameters. 

We will use the same (or similar) ones for v0
"""

# DATA
BATCH_SIZE = 64
AUTO = tf.data.AUTOTUNE
FRAMES_PER_PLAY = 12
INPUT_SHAPE = (FRAMES_PER_PLAY, 120, 54, 10)

# OPTIMIZER
LEARNING_RATE = 1e-4
WEIGHT_DECAY = 1e-5

# TRAINING
EPOCHS = 60

# TUBELET EMBEDDING
PATCH_SIZE = (6, 6, 6)
NUM_PATCHES = (INPUT_SHAPE[0] // PATCH_SIZE[0]) ** 2

# ViViT ARCHITECTURE
LAYER_NORM_EPS = 1e-6
PROJECTION_DIM = 128
NUM_HEADS = 8
NUM_LAYERS = 8

## Step 1: Load and preprocess data

In [4]:
# Load data
[games_df, players_df, plays_df, tracking_df] = load_data()

loaded games df
shape: (136, 9)
-----
loaded players df
shape: (1683, 7)
-----
loaded plays df
shape: (12486, 35)
-----
loading tracking frames...
loaded tracking frames
shape: (12187398, 17)
returning 4 frames


In [5]:
# Preprocess data
tracking_df_clean = preprocess_all_df(plays_df, games_df, players_df, tracking_df)

cleaning plays_df
final plays data shape: (6840, 289)
-----

cleaning games_df
-----

cleaning players_df
-----

cleaning tracking_df
original tracking df shape: (12187398, 17)
unique play and game id combos: (6840, 2)
filtered df shape: (8458178, 17)
number of merge errors: 0
joined plays and tracking dataframes
original tracking shape: (8458178, 17)
merged data shape: (8458178, 20)
-------
joined players and tracking dataframes
original tracking shape: (8458178, 20)
merged data shape: (8458178, 22)
-------
joined games and tracking dataframes
original tracking shape: (8458178, 22)
merged data shape: (8458178, 23)
-------
Old df shape:(8458178, 62)
New df shape:(8458178, 62)
-----



In [14]:
# Get tensors
start_time = time.time()
tensor_list, labels = prepare_4d_tensors(tracking_df_clean, min_frames= 12)
print("processing time:", timedelta(seconds = (time.time() - start_time)))

shape before frame cutoff filter: (8392214, 53)
shape after frame cutoff filter: (2802745, 53)


/Users/jackfriedman/Documents/CS89 Final Project 2/NFL-Big-Data-Bowl/vivit_model/../preprocessing/Preprocessing_v4.py:468: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for frame_id, frame_df in frame_groups:
/Users/jackfriedman/Documents/CS89 Final Project 2/NFL-Big-Data-Bowl/vivit_model/../preprocessing/Preprocessing_v4.py:468: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for frame_id, frame_df in frame_groups:
/Users/jackfriedman/Documents/CS89 Final Project 2/NFL-Big-Data-Bowl/vivit_model/../preprocessing/Preprocessing_v4.py:468: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby

processing time: 0:11:34.766036


## Step 2: Prep data for training (model-specific preprocessing)

### Step 2A: Get min and max yard indices

See www.kaggle.com/competitions/nfl-big-data-bowl-2020/overview/evaluation 

In [21]:
# STEP 0: Round to nearest yard and adjust by 99 (because of the 2020 specifications)
indexed_labels = [round(label) + 99 for label in labels]
min_idx_y = np.min(indexed_labels)
max_idx_y = np.max(indexed_labels)
print('min yardIndex:', min_idx_y)
print('max yardIndex:', max_idx_y)

# STEP 1: CALCULATE NUMBER OF CLASSES (YARDS)
num_classes_y = max_idx_y - min_idx_y + 1
print('num classes:', num_classes_y)

AttributeError: EagerTensor object has no attribute 'astype'. 
        If you are looking for numpy-related methods, please run the following:
        tf.experimental.numpy.experimental_enable_numpy_behavior()
      

### Step 2B: Train-test split

In [17]:
X_train, X_test, y_train, y_test = train_test_split(tensor_list, labels, test_size=0.2, random_state=SEED)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=SEED) # 0.25 x 0.8 = 0.2

### Step 2C: Preprocess and build dataloaders

In [18]:
@tf.function
def preprocess(frames: tf.Tensor, label: tf.Tensor, num_classes_y: int = num_classes_y, min_idx_y: int = min_idx_y):
    """Preprocess the frames tensors and parse the labels."""
    # Preprocess images
    frames = tf.image.convert_image_dtype(
        frames[
            ..., tf.newaxis
        ],  # The new axis is to help for further processing with Conv3D layers
        tf.float32,
    )

    # Preprocess label

    # Ensure min_idx_y is the same type as label_indexed
    min_idx_y = tf.cast(min_idx_y, tf.int32)
    num_classes_y = tf.cast(num_classes_y, tf.int32)

    # Index each label
    label_indexed = tf.cast(tf.round(label), tf.int32) + 99  # Indexing the label

    # One-hot encode the label
    label_one_hot = tf.one_hot(label_indexed - min_idx_y, depth=num_classes_y)

    return frames, label_one_hot


def prepare_dataloader(
    videos: np.ndarray,
    labels: np.ndarray,
    loader_type: str = "train",
    batch_size: int = BATCH_SIZE,
):
    """Utility function to prepare the dataloader."""
    dataset = tf.data.Dataset.from_tensor_slices((videos, labels))

    if loader_type == "train":
        dataset = dataset.shuffle(BATCH_SIZE * 2)

    dataloader = (
        dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
        .batch(batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )
    return dataloader

trainloader = prepare_dataloader(X_train, y_train, "train")
validloader = prepare_dataloader(X_val, y_val, "valid")
testloader = prepare_dataloader(X_test, y_test, "test")

### Step 2D: Define classes for building model

#### (i) Tublet Embedding & Positional Encoding

In [27]:
"""
## Tubelet Embedding

In ViTs, an image is divided into patches, which are then spatially
flattened, a process known as tokenization. For a video, one can
repeat this process for individual frames. **Uniform frame sampling**
as suggested by the authors is a tokenization scheme in which we
sample frames from the video clip and perform simple ViT tokenization.

| ![uniform frame sampling](https://i.imgur.com/aaPyLPX.png) |
| :--: |
| Uniform Frame Sampling [Source](https://arxiv.org/abs/2103.15691) |

**Tubelet Embedding** is different in terms of capturing temporal
information from the video.
First, we extract volumes from the video -- these volumes contain
patches of the frame and the temporal information as well. The volumes
are then flattened to build video tokens.

| ![tubelet embedding](https://i.imgur.com/9G7QTfV.png) |
| :--: |
| Tubelet Embedding [Source](https://arxiv.org/abs/2103.15691) |
"""

# @register_keras_serializable
class TubeletEmbedding(layers.Layer):
    def __init__(self, embed_dim, patch_size, **kwargs):
        super().__init__(**kwargs)
        self.projection = layers.Conv3D(
            filters=embed_dim,
            kernel_size=patch_size,
            strides=patch_size,
            padding="VALID",
        )
        self.flatten = layers.Reshape(target_shape=(-1, embed_dim))

    def call(self, videos):
        projected_patches = self.projection(videos)
        flattened_patches = self.flatten(projected_patches)
        return flattened_patches

In [28]:
"""
## Positional Embedding

This layer adds positional information to the encoded video tokens.
"""

# @register_keras_serializable
class PositionalEncoder(layers.Layer):
    def __init__(self, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim

    def build(self, input_shape):
        _, num_tokens, _ = input_shape
        self.position_embedding = layers.Embedding(
            input_dim=num_tokens, output_dim=self.embed_dim
        )
        self.positions = tf.range(start=0, limit=num_tokens, delta=1)

    def call(self, encoded_tokens):
        # Encode the positions and add it to the encoded tokens
        encoded_positions = self.position_embedding(self.positions)
        encoded_tokens = encoded_tokens + encoded_positions
        return encoded_tokens

#### (ii) Custom metric

In [32]:
class Metric(callbacks.Callback):
    def __init__(self, model, callbacks, dataloader):
        super().__init__()
        self.model = model
        self.callbacks = callbacks
        self.dataloader = dataloader

    def on_train_begin(self, logs=None):
        for callback in self.callbacks:
            callback.on_train_begin(logs)

    def on_train_end(self, logs=None):
        for callback in self.callbacks:
            callback.on_train_end(logs)

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        y_pred_all = []
        y_true_all = []

        # Iterate over the batches of the dataloader
        for X_batch, y_batch in self.dataloader:
            y_pred_batch = self.model.predict_on_batch(X_batch)
            y_pred_all.append(y_pred_batch)
            y_true_all.append(y_batch)

        # Concatenate all batches
        y_pred = np.concatenate(y_pred_all, axis=0)
        y_true = np.concatenate(y_true_all, axis=0)

        # Apply the custom processing to y_pred and y_true
        y_true = np.clip(np.cumsum(y_true, axis=1), 0, 1)
        y_pred = np.clip(np.cumsum(y_pred, axis=1), 0, 1)

        # Calculate the metric
        val_s = ((y_true - y_pred) ** 2).sum(axis=1).sum(axis=0) / (199 * y_true.shape[0])
        logs['val_CRPS'] = val_s

        for callback in self.callbacks:
            callback.on_epoch_end(epoch, logs)


#### (iii) Model architecture 

In [31]:
"""
## Video Vision Transformer

The authors suggest 4 variants of Vision Transformer:

- Spatio-temporal attention
- Factorized encoder
- Factorized self-attention
- Factorized dot-product attention

In this example, we will implement the **Spatio-temporal attention**
model for simplicity. The following code snippet is heavily inspired from
[Image classification with Vision Transformer](https://keras.io/examples/vision/image_classification_with_vision_transformer/).
One can also refer to the
[official repository of ViViT](https://github.com/google-research/scenic/tree/main/scenic/projects/vivit)
which contains all the variants, implemented in JAX.
"""

# Loss function - Continuous Ranked Probability Score
def crps(y_true, y_pred):
    loss = K.mean(K.sum((K.cumsum(y_pred, axis = 1) - K.cumsum(y_true, axis=1))**2, axis=1))/199
    return loss

def create_vivit_classifier(
    tubelet_embedder,
    positional_encoder,
    input_shape=INPUT_SHAPE,
    transformer_layers=NUM_LAYERS,
    num_heads=NUM_HEADS,
    embed_dim=PROJECTION_DIM,
    layer_norm_eps=LAYER_NORM_EPS,
    output_size=num_classes_y,
    l1_reg=0.01,  # L1 regularization factor
    l2_reg=0.01,  # L2 regularization factor
    pdrop=0.3    # Base dropout probability for stochastic depth
):
    # Get the input layer
    inputs = layers.Input(shape=input_shape)
    # Create patches.
    patches = tubelet_embedder(inputs)
    # Encode patches.
    encoded_patches = positional_encoder(patches)

    # Create multiple layers of the Transformer block.
    for l in range(transformer_layers):
        # Compute dropout probabiltiy via stoachstic depth regularization
        depth_drop_prob = l / transformer_layers * pdrop

        # Layer normalization and MHSA
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim // num_heads, dropout=depth_drop_prob
        )(x1, x1)

        # Skip connection
        x2 = layers.Add()([attention_output, encoded_patches])

        # Layer Normalization and MLP
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        x3 = keras.Sequential(
            [
                layers.Dense(units=embed_dim * 4, 
                             activation=tf.nn.gelu,
                             kernel_regularizer=regularizers.l1_l2(l1=l1_reg, l2=l2_reg)),
                layers.Dense(units=embed_dim, 
                             activation=tf.nn.gelu,
                             kernel_regularizer=regularizers.l1_l2(l1=l1_reg, l2=l2_reg)),
            ]
        )(x3)

        # Skip connection
        encoded_patches = layers.Add()([x3, x2])

    # Layer normalization and Global average pooling.
    representation = layers.LayerNormalization(epsilon=layer_norm_eps)(encoded_patches)
    representation = layers.GlobalAvgPool1D()(representation)

    # Classify outputs.
    outputs = layers.Dense(units=output_size, activation="softmax")(representation)

    # Create the Keras model.
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

## Step 3: Train Model

In [33]:
"""
## Train
"""


def run_experiment():
    # Initialize model
    model = create_vivit_classifier(
        tubelet_embedder=TubeletEmbedding(
            embed_dim=PROJECTION_DIM, patch_size=PATCH_SIZE
        ),
        positional_encoder=PositionalEncoder(embed_dim=PROJECTION_DIM),
    )

    # Compile the model with the optimizer, loss function
    # and the metrics.
    optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
    model.compile(
        optimizer=optimizer,
        loss=crps
    )
    # To implement early stopping, use Keras callbacks when fitting the model
    early_stopping = callbacks.EarlyStopping(
        monitor='val_loss', patience=10, restore_best_weights=True
    )

    # Instantiate the custom Metric
    metric = Metric(model, [early_stopping], validloader)

    

    # Train the model
    print("training now...")
    start_time = time.time()
    # Train the model
    history = model.fit(
        trainloader,
        epochs=EPOCHS,
        validation_data=validloader,
        callbacks=[metric]  # Include the custom Metric here
    )
    print("finished training")
    print("training time:", timedelta(seconds = (time.time() - start_time)))

    # Print metrics
    train_crps = model.evaluate(trainloader)
    print(f"Test CRPS: {round(train_crps, 4)}")
    
    val_crps = model.evaluate(validloader)
    print(f"Val CRPS: {round(val_crps, 4)}")

    test_crps = model.evaluate(testloader)
    print(f"Test CRPS: {round(test_crps, 4)}")

    return model


model = run_experiment()

training now...
Epoch 1/60


In [ ]:
# Save model
model.save('vivit_v1_model.h5')

In [ ]:
# # Load model
# model_pickled = load_model('vivit_v1_model.h5', custom_objects={'TubeletEmbedding': TubeletEmbedding, 
#                                                                 'PositionalEncoder': PositionalEncoder})